In [1]:
from utils import *

# load data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/")

Extracting tmp/train-images-idx3-ubyte.gz
Extracting tmp/train-labels-idx1-ubyte.gz
Extracting tmp/t10k-images-idx3-ubyte.gz
Extracting tmp/t10k-labels-idx1-ubyte.gz


In [2]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, name="outputs")

In [3]:
# get the weights from the layers
W1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
W2 = tf.get_default_graph().get_tensor_by_name("outputs/kernel:0")

scale = 0.01 # regularization weight

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name="xentropy")
    # calculate conventionaln loss
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    # add sum of absolute values for reg losses
    reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2))
    # loss is base loss + scale * reg loss
    loss = tf.add(base_loss, scale * reg_losses, name="loss")

In [4]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter("to_tensorboard/11_15_reg_man", tf.get_default_graph())

In [5]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict= {X:X_batch, y:y_batch})
        accuracy_val = accuracy.eval(feed_dict={X:mnist.validation.images, y:mnist.validation.labels})
        print(epoch, "Validation accuracy:", accuracy_val)
    accuracy_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    print("Final test accuracy:", accuracy_test)
    
file_writer.close()

0 Validation accuracy: 0.7966
1 Validation accuracy: 0.789
2 Validation accuracy: 0.6846
3 Validation accuracy: 0.5932
4 Validation accuracy: 0.5956
5 Validation accuracy: 0.6418
6 Validation accuracy: 0.6756
7 Validation accuracy: 0.703
8 Validation accuracy: 0.7244
9 Validation accuracy: 0.737
10 Validation accuracy: 0.7508
11 Validation accuracy: 0.758
12 Validation accuracy: 0.7626
13 Validation accuracy: 0.7674
14 Validation accuracy: 0.7716
15 Validation accuracy: 0.7754
16 Validation accuracy: 0.7792
17 Validation accuracy: 0.7798
18 Validation accuracy: 0.785
19 Validation accuracy: 0.7874
Final test accuracy: 0.7793
